In [ ]:
import itertools
import tensorflow as tf
import numpy as np
from packaging import version

from sklearn.metrics import roc_curve, auc, precision_score, recall_score, f1_score

from matplotlib import pyplot as plt



print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

# Weight initializers for the Generator network
WEIGHT_INIT = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.2)

In [ ]:
def plot_roc_curve(fpr, tpr, name_model):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.savefig(name_model+'_roc_curve.png')
    plt.show()
    plt.clf()
    


''' calculate the auc value for labels and scores'''
def roc(labels, scores, name_model):
    """Compute ROC curve and ROC area for each class"""

    # True/False Positive Rates.
    fpr, tpr, threshold = roc_curve(labels, scores)
    print("threshold: ", threshold)
    roc_auc = auc(fpr, tpr)
    # get a threshold that perform very well.
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    # draw plot for ROC-Curve
    plot_roc_curve(fpr, tpr, name_model)
    
    return roc_auc, optimal_threshold

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        c_map=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=c_map)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(title+'_cm.png')
    plt.show()
    plt.clf()

In [ ]:
if __name__ == "__main__":
    
    '''
    Model for computer-vision-based Printed Circuit Board (PCB).
    analysis dataset used for classification of defects.
    '''
    
    
    # run the function here
    nameModel= "pcb"
    print("start: ", nameModel)
    """ Set Hyper parameters """
    batch_size = 25
    num_epochs = 2
    
    IMG_H = 110
    IMG_W = 42
    IMG_C = 3  ## Change this to 1 for grayscale.

    # set dir of files
    train_data_path = "image_dataset/training_dataset"
    test_data_path = "image_dataset/evaluation_dataset"

    input_shape = (IMG_W, IMG_H, IMG_C)
    # print(input_shape)

    # set input 
    inputs = tf.keras.layers.Input(input_shape, name="input_1")

    
 
    train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.3) # don't use rescale = 1./255

    train_generator = train_data_gen.flow_from_directory(train_data_path,
                                                     target_size=(IMG_W,IMG_H),
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     class_mode='categorical',
                                                     subset='training')

    validation_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.3)

    validation_generator =  validation_data_gen.flow_from_directory(train_data_path,
                                                                target_size=(IMG_W,IMG_H),
                                                     batch_size=batch_size,
                                                                class_mode='categorical',
                                                                subset='validation')  

    
    class_names = train_generator.class_names
    image_batch_train, labels_batch_train = next(iter(train_generator))
    print(image_batch_train.shape)
    print(labels_batch_train.shape)


    
    data_augmentation = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
      tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    ])

    num_classes = 8

    our_model = tf.keras.models.Sequential([
      tf.keras.layers.Rescaling(1./255, input_shape=(IMG_H, IMG_W, IMG_C)),
      tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(num_classes)
    ])

    our_model.summary()
    
    base_learning_rate = 0.0001
    
    our_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

    fit_history = our_model.fit(
        train_generator,
        epochs=num_epochs,
        validation_data=validation_generator
    )
    
    
    
    acc = fit_history.history['accuracy']
    val_acc = fit_history.history['val_accuracy']
    loss = fit_history.history['loss']
    val_loss = fit_history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')

    plt.figure()

    plt.plot(epochs, loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')

    plt.legend()

    plt.show()
    
    
    
    """
    Evaluation Area
    """


